In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
_query = """
SELECT
  run,
  trim('"' FROM seq::text )::int as seq,
  trim('"' FROM block::text )::int as block,
  trim('"' FROM min::text )::int as min,
  --e0,
  e1::decimal/100000000 as batch_sec, -- time in sec as
  --e2
  ts::text::time

FROM
  (SELECT
    run,
    e->'ts' as ts,
    e->'seq' as seq,
    e->'log'->'height' as block,
    e->'log'->'min' as min,
    --e->'log'->'event' as event,
    e->'log'->'event'->0 as e0,
    e->'log'->'event'->1 as e1,
    e->'log'->'event'->2 as e2
  FROM
    fnode0.simtest) as s
WHERE
  e2::text like '%RUNTIME%'
"""

_order = """
ORDER BY
  run, seq ;
"""

def query(i):
    return _query + ' AND run = %s ' % i + _order

connection = pg.connect("host='localhost' dbname=load user=load password='load'")

In [3]:
log_runs = pd.read_sql_query('SELECT * from log_runs;', con=connection)
log_runs

,id,label,ts
0,1,v5_oldcode_Jan30_2_drop_line,2019-01-31 20:01:40.916787
1,2,v6_newcode_Jan30_2_drop_line,2019-01-31 20:17:18.531736


In [4]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
for i, n in log_runs['label'].items():
    r = log_runs['id'][i]
    df = pd.read_sql_query(query(r), con=connection)
    data.append(
        go.Scatter(x=df['ts'], y=df['batch_sec'], name=n.replace('test_',''))
    )

py.iplot(data, filename='holdingqueueclearing')